In [1]:
import pandas as pd
import os
import numpy as np
import sys
import cantera as ct

In [3]:
df = pd.read_csv('sim_data/rtol_1e-10_atol_1e-20_data.csv')
table_names = ['Ratio', 'SYNGAS Selec', 'SYNGAS Yield', 'CO Selectivity', 'CO % Yield', 'H2 Selectivity', 
               'H2 % Yield', 'CH4 Conversion', 'H2O+CO2 Selectivity', 'H2O+CO2 yield', 'Exit Temp', 'Peak Temp',
               'Dist to peak temp', 'O2 Conversion', 'Max CH4 Conv', 'Dist to 50 CH4 Conv', 'Half Temp Pos']
data_to_csv = []
for i in range(len(df)):
    sens_data = calc(df.iloc[i,1:], type='sens')
    data_to_csv.append(sens_data)
df_to_csv = pd.DataFrame(np.array(data_to_csv), columns=table_names)
df_to_csv.to_csv('sim_data/therm_sens_ref.csv')

In [2]:
def calc(data, type='sens'):
    """
    Calculate properties of interest from the raw data
    :param data: the data
    :param type: 'sens' for sensitivity analyses
                 'output' for saving the output csv
                 'ratio' for plotting
    :return:
    """
    ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv, sens_id = data

    ch4_depletion = ch4_in - ch4_out
    reference_ch4_conv = ch4_depletion / ch4_in 
    # negative sensitivity is higher selectivity
    reference_h2_sel = h2_out / (ch4_depletion * 2)  # Sensitivity definition 5: H2 selectivity
    if reference_h2_sel <= 0:
        reference_h2_sel = 1.0e-15  # selectivity can't be 0

    reference_co_sel = co_out / ch4_depletion  # Sensitivity definition 3: CO selectivity
    if reference_co_sel <= 0:
        reference_co_sel = 1.0e-15  # selectivity can't be 0

    reference_syngas_selectivity = reference_co_sel + reference_h2_sel  # Sensitivity definition 1: SYNGAS selectivity

    reference_syngas_yield = reference_syngas_selectivity * reference_ch4_conv  # Sensitivity definition 2: SYNGAS yield
    if reference_syngas_yield <= 0:
        reference_syngas_yield = 1.0e-15  # yield can't be 0

    reference_co_yield = co_out / ch4_in  # Sensitivity definition 4: CO % yield
    # reference_co_yield = reference_co_sel * reference_ch4_conv

    reference_h2_yield = h2_out / (2 * ch4_in)  # Sensitivity definition 6: H2 % yield
    # reference_h2_yield = reference_h2_sel * reference_ch4_conv

    # Sensitivity definition 8: H2O + CO2 selectivity
    reference_h2o_sel = h2o_out / (ch4_depletion * 2)
    reference_co2_sel = co2_out / ch4_depletion
    if reference_h2o_sel <= 0:
        reference_h2o_sel = 1.0e-15  # H2O selectivity can't be 0
    if reference_co2_sel <= 0:
        reference_co2_sel = 1.0e-15  # CO2 selectivity can't be 0
    reference_full_oxidation_selectivity = reference_h2o_sel + reference_co2_sel

    # Sensitivity definition 9: H2O + CO2 yield
    reference_full_oxidation_yield = reference_full_oxidation_selectivity * reference_ch4_conv
    
    sens_property = [ratio, reference_syngas_selectivity, reference_syngas_yield, reference_co_sel, reference_co_yield, reference_h2_sel, 
                     reference_h2_yield, reference_ch4_conv, reference_full_oxidation_selectivity, reference_full_oxidation_yield, 
                     reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, 
                     reference_dist_to_50_ch4_conv, sens_id]
    
    ref_data = [ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, 
                reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv, sens_id]
    if type is 'sens':
        return sens_property
    elif type is 'ratio':
        return reference_co_sel, reference_h2_sel, reference_ch4_conv, reference_exit_temp, reference_o2_conv, reference_co2_sel, reference_h2o_sel
    elif type is 'gas_data':
        return ratio, reference
    else:
        return ref_data

In [6]:
x = pd.read_csv('sim_data/therm_sens_ref.csv')
x

,Unnamed: 0,Ratio,SYNGAS Selec,SYNGAS Yield,CO Selectivity,CO % Yield,H2 Selectivity,H2 % Yield,CH4 Conversion,H2O+CO2 Selectivity,H2O+CO2 yield,Exit Temp,Peak Temp,Dist to peak temp,O2 Conversion,Max CH4 Conv,Dist to 50 CH4 Conv,Half Temp Pos
0,0,0.6,1.218245,0.429478,0.513827,0.181144,0.704418,0.248335,0.352539,0.777490,0.274096,1906.434140,1906.434140,70.00,0.437634,-0.045447,10.76,1045.0
1,1,0.7,1.363151,0.471536,0.547002,0.189217,0.816148,0.282319,0.345916,0.636165,0.220060,1640.524185,1640.524185,70.00,0.440544,-0.046779,10.79,1045.0
2,2,0.8,1.475504,0.478268,0.572391,0.185534,0.903112,0.292734,0.324139,0.524381,0.169972,1401.034078,1412.331317,14.31,0.420388,-0.043833,10.85,1045.0
3,3,0.9,1.531676,0.355960,0.569477,0.132346,0.962199,0.223614,0.232399,0.468249,0.108821,1226.321666,1274.989363,14.00,0.314992,-0.034853,11.22,1045.0
4,4,1.0,1.502016,0.223539,0.524843,0.078110,0.977173,0.145429,0.148826,0.497822,0.074089,1175.094856,1219.572028,14.59,0.226291,-0.024950,11.86,1045.0
5,5,1.1,1.464326,0.154471,0.481967,0.050843,0.982359,0.103629,0.105490,0.535422,0.056482,1153.554925,1188.623855,15.15,0.180188,-0.019185,12.57,1045.0
6,6,1.2,1.434323,0.113884,0.449131,0.035661,0.985192,0.078223,0.079399,0.565338,0.044887,1139.088367,1166.514852,15.63,0.150524,-0.015762,13.38,1045.0
7,7,1.3,1.412488,0.087811,0.425442,0.026449,0.987045,0.061362,0.062168,0.587090,0.036498,1127.526469,1148.951726,16.07,0.129279,-0.013482,14.31,1045.0
8,8,1.4,1.397950,0.070061,0.409575,0.020527,0.988375,0.049535,0.050117,0.601546,0.030148,1117.592731,1134.222383,16.47,0.113152,-0.011834,15.40,1045.0
9,9,1.6,1.386527,0.048195,0.396357,0.013777,0.990171,0.034418,0.034760,0.612808,0.021301,1100.665637,1110.186204,17.24,0.090208,-0.009570,18.45,1045.0
